<picture>
<a href="https://sambanova.ai/"\>
<source media="(prefers-color-scheme: dark)" srcset="../../images/SambaNova-light-logo-1.png" height="60">
<img alt="SambaNova logo" src="../../images/SambaNova-dark-logo-1.png" height="60">
</picture>
</a>

# Agentops Agent Tracking with SambaNova 

Adapted from the same [Agentops example](https://docs.agentops.ai/v1/examples/multi_agent).

Integrating AgentOps with SambaNova Cloud enhances AI agent development by providing real-time monitoring, debugging, and cost management for LLM inference. AgentOps offers session replay analytics, time-travel debugging, and API latency tracking, helping developers optimize performance and troubleshoot issues efficiently. 

You'll need first to import the necessary modules. Please, create a `.env` file at the repository root and include `SAMBANOVA_URL` and `SAMBANOVA_API_KEY` after creating your [SambaNova Cloud API key](https://cloud.sambanova.ai/apis). You'll also need to add the `AGENTOPS_API_KEY`, so follow the steps in [this link.](https://app.agentops.ai/settings/projects)

Then, create a venv using the framework that you prefer and install the dependencies in the `requirements.txt` located in the current folder.


In [ ]:
import os
import agentops
from agentops import track_agent
from openai import OpenAI
from IPython.display import display, Markdown

In [ ]:
from dotenv import load_dotenv

load_dotenv('../../.env',override=True)

sambanova_base_url = os.getenv('SAMBANOVA_URL')
sambanova_api_key = os.getenv('SAMBANOVA_API_KEY')
agentops_api_key = os.getenv("AGENTOPS_API_KEY")

import logging
logging.basicConfig(
    level=logging.DEBUG
)  # this will let us see that calls are assigned to an agent

In [ ]:
agentops.init(agentops_api_key, default_tags=["multi-agent-notebook"])
sambanova_client = OpenAI(base_url=sambanova_base_url, api_key=sambanova_api_key)

In [ ]:
@track_agent(name="qa")
class QaAgent:
    def completion(self, prompt: str):
        res = sambanova_client.chat.completions.create(
            model="Meta-Llama-3.1-70B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a qa engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content


@track_agent(name="engineer")
class EngineerAgent:
    def completion(self, prompt: str):
        res = sambanova_client.chat.completions.create(
            model="Meta-Llama-3.1-70B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": "You are a software engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content

In [ ]:
qa = QaAgent()
engineer = EngineerAgent()

In [ ]:
generated_func = engineer.completion("python function to test prime number")

In [ ]:
display(Markdown("```python\n" + generated_func + "\n```"))

In [ ]:
generated_test = qa.completion(
    "Write a python unit test that test the following function: \n " + generated_func
)

In [ ]:
display(Markdown("```python\n" + generated_test + "\n```"))

In [ ]:
res = sambanova_client.chat.completions.create(
    model="Meta-Llama-3.1-70B-Instruct",
    messages=[
        {"role": "system", "content": "You are not a tracked agent"},
        {"role": "user", "content": "Say hello"},
    ],
)
res.choices[0].message.content

Look at the **Session Replay** in the following cell output. Click on it and log in to AgentOps. You'll see several options related to your LLM runs and stats.

In [ ]:
agentops.end_session("Success")
# ...
# 🖇 AgentOps: Session Stats - Duration: Xs | Cost: $X | LLMs: 3 | Tools: 0 | Actions: 0 | Errors: 0
# 🖇 AgentOps: Session Replay: <<AGENTOPS_URL>>